In [1]:
#Import the required libraries
!pip install keras-tuner
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import pathlib, os, random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow as tf
from PIL import Image
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import time
import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler


import cv2
import matplotlib.pyplot as plt 
import seaborn as sns
import os
from PIL import Image
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
from keras.utils import np_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#Understanding the files in different data directories
bad_data = os.listdir('/content/drive/MyDrive/Ethiopia/Bad')
print("Bad coffee beans: ",bad_data[:10])

good_data = os.listdir('/content/drive/MyDrive/Ethiopia/Good')
print("Good coffee beans: ",good_data[:10])

Bad coffee beans:  ['IMG_6040_5.jpg', 'IMG_6038_7.jpg', 'IMG_6036_5.jpg', 'IMG_6036_2.jpg', 'IMG_6042_5.jpg', 'IMG_6039_9.jpg', 'IMG_6040_8.jpg', 'IMG_6040_3.jpg', 'IMG_6042_1.jpg', 'IMG_6039_3.jpg']
Good coffee beans:  ['IMG_5968_6.jpg', 'IMG_5974_7.jpg', 'IMG_5966_7.jpg', 'IMG_5967_4.jpg', 'IMG_5968_9.jpg', 'IMG_5970_3.jpg', 'IMG_5976_1.jpg', 'IMG_5971_6.jpg', 'IMG_5968_1.jpg', 'IMG_5969_2.jpg']


In [3]:
data = []
labels = []
for img in bad_data:
    try:
        img_read = plt.imread('/content/drive/MyDrive/Ethiopia/Bad' + "/" + img)
        img_resize = cv2.resize(img_read, (256,256))
        img_array = img_to_array(img_resize)
        data.append(img_array)
        labels.append(1)
    except:
        None
        
for img in good_data:
    try:
        img_read = plt.imread('/content/drive/MyDrive/Ethiopia/Good' + "/" + img)
        img_resize = cv2.resize(img_read, (256,256))
        img_array = img_to_array(img_resize)
        data.append(img_array)
        labels.append(0)
    except:
        None

In [4]:
image_data = np.array(data)
labels = np.array(labels)

idx = np.arange(image_data.shape[0])
np.random.shuffle(idx)
image_data = image_data[idx]
labels = labels[idx]

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(image_data, labels, test_size = 0.2, random_state = 101)

y_train = np_utils.to_categorical(y_train, num_classes = 2)
y_test = np_utils.to_categorical(y_test, num_classes = 2)

print(f'SHAPE OF TRAINING IMAGE DATA : {x_train.shape}')
print(f'SHAPE OF TESTING IMAGE DATA : {x_test.shape}')
print(f'SHAPE OF TRAINING LABELS : {y_train.shape}')
print(f'SHAPE OF TESTING LABELS : {y_test.shape}')

SHAPE OF TRAINING IMAGE DATA : (844, 256, 256, 3)
SHAPE OF TESTING IMAGE DATA : (211, 256, 256, 3)
SHAPE OF TRAINING LABELS : (844, 2)
SHAPE OF TESTING LABELS : (211, 2)


In [6]:
!pip install hyperactive==0.3.1
import hyperactive as hyp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
search_config = {
    "keras.compile.0": {"loss": ["binary_crossentropy"], "optimizer": ["adam"]},
    "keras.fit.0": {"epochs": [5], "batch_size": [32], "verbose": [2]},
    "keras.layers.Conv2D.1": {
        "filters": [32, 64, 128],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
        "input_shape": [(256,256,3)],
    },
    "keras.layers.MaxPooling2D.2": {"pool_size": [(2, 2)]},
    "keras.layers.Conv2D.3": {
        "filters": [16, 32, 64, 128],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
    },
    "keras.layers.MaxPooling2D.4": {"pool_size": [(2, 2)]},
    "keras.layers.Flatten.5": {},
    "keras.layers.Dense.6": {"units": range(30, 100, 10), "activation": ["relu"]},
    "keras.layers.Dropout.7": {"rate": list(np.arange(0.2, 0.8, 0.2))},
    "keras.layers.Dense.8": {"units": [2], "activation": ["softmax"]},
}
start_point = {
    "keras.compile.0": {"loss": ["binary_crossentropy"], "optimizer": ["adam"]},
    "keras.fit.0": {"epochs": [5], "batch_size": [32], "verbose": [2]},
    "keras.layers.Conv2D.1": {
        "filters": [64],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
        "input_shape": [(256,256,3)],
    },
    "keras.layers.MaxPooling2D.2": {"pool_size": [(2, 2)]},
    "keras.layers.Conv2D.3": {
        "filters": [32],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
    },
    "keras.layers.MaxPooling2D.4": {"pool_size": [(2, 2)]},
    "keras.layers.Flatten.5": {},
    "keras.layers.Dense.6": {"units": [30], "activation": ["relu"]},
    "keras.layers.Dropout.7": {"rate": [0.2]},
    "keras.layers.Dense.8": {"units": [2], "activation": ["softmax"]},
}


#https://pypi.org/project/hyperactive/0.3.1/ 
Optimizer = hyp.ParticleSwarmOptimizer(search_config, n_iter=3,warm_start=start_point, metric="accuracy", n_jobs=1, cv=3, verbosity=1, random_state=42, n_part=3, w=0.85, c_k=1.9, c_s=2.05)

In [ ]:
import time
t1 = time.time()
# search best hyperparameter for given data
h = Optimizer.fit(x_train, y_train)

t2 = time.time()
print("time: {}".format(t2-t1))

# predict from test data
Optimizer.predict(x_test)

# calculate accuracy score
score = Optimizer.score(x_test, y_test)

print("test score: {}".format(score))

Epoch 1/5
27/27 - 15s - loss: 74.4276 - accuracy: 0.5972 - 15s/epoch - 553ms/step
Epoch 2/5
27/27 - 2s - loss: 0.4726 - accuracy: 0.7524 - 2s/epoch - 71ms/step
Epoch 3/5
27/27 - 2s - loss: 0.4118 - accuracy: 0.8614 - 2s/epoch - 70ms/step
Epoch 4/5
27/27 - 2s - loss: 0.4006 - accuracy: 0.8945 - 2s/epoch - 71ms/step
Epoch 5/5
27/27 - 2s - loss: 0.3207 - accuracy: 0.9265 - 2s/epoch - 71ms/step
27/27 [==============================] - 1s 35ms/step - loss: 0.2798 - accuracy: 0.9751


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/5
27/27 - 2s - loss: 374.1037 - accuracy: 0.5581 - 2s/epoch - 80ms/step
Epoch 2/5
27/27 - 1s - loss: 0.6784 - accuracy: 0.7204 - 1s/epoch - 41ms/step
Epoch 3/5
27/27 - 1s - loss: 0.5571 - accuracy: 0.8081 - 1s/epoch - 41ms/step
Epoch 4/5
27/27 - 1s - loss: 0.4661 - accuracy: 0.8543 - 1s/epoch - 41ms/step
Epoch 5/5
27/27 - 1s - loss: 0.4145 - accuracy: 0.8969 - 1s/epoch - 41ms/step
27/27 [==============================] - 1s 26ms/step - loss: 0.3462 - accuracy: 0.9254
Epoch 1/5
27/27 - 3s - loss: 60.3998 - accuracy: 0.6991 - 3s/epoch - 101ms/step
Epoch 2/5
27/27 - 2s - loss: 0.4360 - accuracy: 0.8922 - 2s/epoch - 65ms/step
Epoch 3/5
27/27 - 2s - loss: 0.3977 - accuracy: 0.9206 - 2s/epoch - 65ms/step
Epoch 4/5
27/27 - 2s - loss: 0.3314 - accuracy: 0.9491 - 2s/epoch - 65ms/step
Epoch 5/5
27/27 - 2s - loss: 0.2872 - accuracy: 0.9633 - 2s/epoch - 65ms/step
27/27 [==============================] - 1s 38ms/step - loss: 0.3414 - accuracy: 0.9467
Epoch 1/5
27/27 - 7s - loss: 160.1965 - 

 33%|███▎      | 1/3 [01:10<02:20, 70.05s/it]

Epoch 1/5
27/27 - 2s - loss: 133.8815 - accuracy: 0.5533 - 2s/epoch - 61ms/step
Epoch 2/5
27/27 - 1s - loss: 0.4170 - accuracy: 0.7950 - 1s/epoch - 41ms/step
Epoch 3/5
27/27 - 1s - loss: 0.5859 - accuracy: 0.8803 - 1s/epoch - 41ms/step
Epoch 4/5
27/27 - 1s - loss: 0.6953 - accuracy: 0.5000 - 1s/epoch - 41ms/step
Epoch 5/5
27/27 - 1s - loss: 0.6951 - accuracy: 0.5000 - 1s/epoch - 41ms/step
27/27 [==============================] - 1s 24ms/step - loss: 0.6948 - accuracy: 0.5000
Epoch 1/5
27/27 - 2s - loss: 49.6148 - accuracy: 0.6659 - 2s/epoch - 89ms/step
Epoch 2/5
27/27 - 2s - loss: 0.4399 - accuracy: 0.8555 - 2s/epoch - 71ms/step
Epoch 3/5
27/27 - 2s - loss: 0.3265 - accuracy: 0.8981 - 2s/epoch - 71ms/step
Epoch 4/5
27/27 - 2s - loss: 0.2651 - accuracy: 0.9301 - 2s/epoch - 71ms/step
Epoch 5/5
27/27 - 2s - loss: 0.2160 - accuracy: 0.9502 - 2s/epoch - 71ms/step
27/27 [==============================] - 1s 36ms/step - loss: 0.1349 - accuracy: 0.9727
Epoch 1/5
27/27 - 2s - loss: 24.4301 - ac

 67%|██████▋   | 2/3 [01:43<00:48, 48.59s/it]

Epoch 1/5
27/27 - 2s - loss: 56.0617 - accuracy: 0.6671 - 2s/epoch - 90ms/step
Epoch 2/5
27/27 - 2s - loss: 0.4907 - accuracy: 0.8021 - 2s/epoch - 71ms/step
Epoch 3/5
27/27 - 2s - loss: 0.3980 - accuracy: 0.8863 - 2s/epoch - 71ms/step
Epoch 4/5
27/27 - 2s - loss: 0.3625 - accuracy: 0.9171 - 2s/epoch - 71ms/step
Epoch 5/5
27/27 - 2s - loss: 0.3092 - accuracy: 0.9289 - 2s/epoch - 71ms/step
